In [ ]:
import os
import random
import torch
import pickle
import copy
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from numpy.random import randint
import scipy.stats as stats

# save images
from torchvision.utils import save_image
img_save_path = './plots/'
os.makedirs(img_save_path, exist_ok=True)


from deap import base, creator, tools, algorithms

from keras.models import Model
from keras.layers import Dense, Dropout
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.utils.image_utils import load_img, img_to_array
import tensorflow as tf
from PIL import Image

from NIMA.utils.score_utils import mean_score
from high_resolution import upscale
from CAN.parameters import *
from CAN.model_CAN_16_9 import * 

In [ ]:
# Set random seed for reproducibility
manualSeed = 7
#manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
np.random.seed(manualSeed)


# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [ ]:
### load Generator model

# Create the generator
netG = Generator(ngpu).to(device)

# Setup Adam optimizers for G
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

# Path to model parameters
PATH_GEN = './CAN/models/GEN_16_9.pth'

# load checkpoint
checkpoint = torch.load(PATH_GEN, map_location=torch.device('cpu'))
netG.load_state_dict(checkpoint['model_state_dict'])
optimizerG.load_state_dict(checkpoint['optimizer_state_dict'])

# set model to evaluation
netG.eval()

In [ ]:
### load Evaluation model

netEval = InceptionResNetV2(input_shape=(None, None, 3), include_top=False, pooling='avg', weights=None)
x = Dropout(0.75)(netEval.output)
x = Dense(10, activation='softmax')(x)
netEval = Model(netEval.input, x)
netEval.load_weights('./NIMA/weights/inception_resnet_weights.h5')

In [ ]:
### Fitness functions

# automatic evaluation

def AutomaticEvaluation(img_path):
    
    with tf.device('/CPU:0'):
        # load image, unfortunately from path due to problems in converting torch.tensor to PilImage
        target_size = (224, 224)
        image = load_img(img_path, target_size=target_size)
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        image = preprocess_input(image)

        # evaluate the image
        score = netEval.predict(image, batch_size=1, verbose=0)[0]
        score = mean_score(score)

    return score

def fitnessAutomaticEvaluation(ind):

    # part 1: create image
    
    with torch.no_grad():
        ind = torch.tensor(ind)
        ind = ind.view(1,nz,1,1)
        image = netG(ind.float()).detach().cpu()

        # not really sexy, but when directly converting to PilImage, image is slightly changed!
        # delete after reading it in, no need to keep
        save_image(image.data,"current_individual.png", normalize=True)
    
    
    # part 2: evluate image and delete it

    score = AutomaticEvaluation("current_individual.png")
    os.remove("current_individual.png")


    return score, # deap needs the comma



# collaborative interactive evaluation

def fitnessInteractiveEvaluation(ind, n = 5):   

    ## Input from n participants

    scores = []

    for i in range(n):
        scores.append(int(input('Please enter the fitness rating of participant %d' % (i+1))))
    
    return np.mean(scores), # deap needs the comma (iterable tuple)



In [ ]:
### DEAP setup

# maximation, one objective
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# set up individuals, random normal distribution
toolbox = base.Toolbox()
toolbox.register("attribute", np.random.normal)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                 toolbox.attribute, n=nz)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

### set up mating, mutation, selection

# from experiments with recombination, uniform with indpb = .25 promising
toolbox.register("mate", tools.cxUniform, indpb = 0.25)
# only used in local search, thus can stay as generell setup
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=1/100)
# next two overwritten in local search
toolbox.register("select", tools.selStochasticUniversalSampling)
toolbox.register("evaluate", fitnessInteractiveEvaluation)

In [ ]:
### save some statistics of each generation
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("max", np.max)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("std", np.std)

In [ ]:
# local search instead of mutation

def local_Search(genes, NGEN = 100):

    pop = toolbox.population(n=1)
    pop[0] = (creator.Individual(genes))
    ref = copy.deepcopy(pop[0])

    # switch to selBest, mutation is only used here and can be kept
    toolbox.register("select", tools.selBest)
    # and switch to automatic evaluation
    toolbox.register("evaluate", fitnessAutomaticEvaluation) 

    offspring = algorithms.eaMuPlusLambda(pop, toolbox, mu=1, lambda_=1, 
        cxpb=0, mutpb=1, ngen=NGEN, stats=stats, verbose=False)
    
    # switch back
    toolbox.register("select", tools.selStochasticUniversalSampling)
    toolbox.register("evaluate", fitnessInteractiveEvaluation)

    same = False
    if all(x == y for x, y in zip(ref, offspring[0][0])):
        same = True

    return offspring[0][0], same

In [ ]:
# diversity preservation metric to avoid too close individuals:

def preserve_Diversity(pop, MU, threshold = 25):
    invalid_inds = []
    dist = []
    # Threshold ~ 5-30, arbitrary

    for i in range(MU):
        for j in range(MU):
            if j > i:
                dist.append(sum([abs(x-y) for x, y in zip(pop[i], pop[j])]))
                if sum([abs(x-y) for x, y in zip(pop[i], pop[j])]) < threshold:
                    invalid_inds.append(i)

    return np.unique(invalid_inds).tolist(), dist 

In [ ]:
def complex_Evolution(MU = 15, crossP = 0.5, mutP = 0.5, NGEN = 25, checkpoint = None):

    ## use checkpoint for interactive to not lose results, can last weeks!
    if checkpoint:
        # a file name has been given, then load the data from the file
        with open(checkpoint, "rb") as cp_file:
            cp = pickle.load(cp_file)
        pop = cp["population"]
        start_gen = cp["generation"] + 1
        hof = cp["halloffame"]
        logbook = cp["logbook"]
        random.setstate(cp["rndstate"])

        for gen in logbook:
            print(gen)

    else:
        # create first generation
        genes = torch.randn(MU, nz, 1, 1, device=device)

        #log and hof
        logbook = tools.Logbook()
        logbook.header = ['gen', 'nevals', 'imm'] + (stats.fields if stats else [])
        hof = tools.HallOfFame(maxsize=20)
        start_gen = 0
        
        # population
        pop = toolbox.population(n=MU)
        # genes replacing the pop
        for i in range(MU):
            ind = genes[i].tolist()
            #flatten lists
            ind = [item for sublist in ind for item in sublist]
            ind = [item for sublist in ind for item in sublist]
            pop[i] = (creator.Individual(ind))


        ## evaluation of first generation
        invalid_ind = [ind for ind in pop if not ind.fitness.valid]

        # interactive evaluation: Save all images upscaled that are to be evaluated
        for i, img in enumerate(invalid_ind):
            # save images
            with torch.no_grad():
                new_genes = torch.tensor(img)
                new_genes = new_genes.view(1,nz,1,1)
                fake = netG(new_genes.float()).detach().cpu()
                save_image(fake.data, img_save_path + '/Evaluation/ind_0_%d.png' % (i), normalize=True)


            upscale(img_path = img_save_path + '/Evaluation/ind_0_%d.png' % (i), scale = 8)
            os.remove(img_save_path + '/Evaluation/ind_0_%d.png' % (i))


        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # get stats from first generation
        record = stats.compile(pop)
        logbook.record(gen=start_gen, nevals=len(invalid_ind), imm=0, **record)
        hof.update(pop)
        print(logbook.stream)

        ## save results in checkpoint in case of problems 
        cp = dict(population=pop, generation=start_gen, halloffame=hof,
                        logbook=logbook, rndstate=random.getstate())

        with open("checkpoint_%d.pkl" % (start_gen), "wb") as cp_file:
            pickle.dump(cp, cp_file)



    for i in range(start_gen, NGEN):

        # save images
        with torch.no_grad():
            new_genes = torch.tensor(pop)
            new_genes = new_genes.view(MU,nz,1,1)
            fake = netG(new_genes.float()).detach().cpu()
            save_image(fake.data, img_save_path + '/evolution_collaborative_%d.png' % (i), normalize=True, nrow = 5)


        # let the algorithm run for one round only, then plot the results again
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < crossP:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Apply mutation on the offspring: Here memetic algorithm using local search!
        for mutant in range(len(offspring)):
            if random.random() < mutP:
                offspring[mutant], same = local_Search(offspring[mutant])
                if same == False:
                    del offspring[mutant].fitness.values


        # Preserve diversity, for novelty search + exploration + against user fatigue
        boring_inds, _ = preserve_Diversity(offspring, MU)
        for ind in boring_inds:

            ## to check the measure:
            with torch.no_grad():
                new_genes = torch.tensor(offspring[ind])
                new_genes = new_genes.view(1,nz,1,1)
                fake = netG(new_genes.float()).detach().cpu()
                save_image(fake.data, img_save_path + '/evolution_collaborative_%d_replacedInd_%d.png' % (i+1, ind), normalize=True, nrow = 5)

            # replace by random immigrant if to close to other individuals
            offspring[ind] = toolbox.population(n=1)[0]
            # and go through local search directly
            offspring[ind],_ = local_Search(offspring[ind])
            del offspring[ind].fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]

        for j, img in enumerate(invalid_ind):
            # save images
            with torch.no_grad():
                new_genes = torch.tensor(img)
                new_genes = new_genes.view(1,nz,1,1)
                fake = netG(new_genes.float()).detach().cpu()
                save_image(fake.data, img_save_path + '/Evaluation/ind_%d_%d.png' % (i+1, j), normalize=True)


            upscale(img_path = img_save_path + '/Evaluation/ind_%d_%d.png' % (i+1, j), scale = 8)
            os.remove(img_save_path + '/Evaluation/ind_%d_%d.png' % (i+1, j))


        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # The population is entirely replaced by the offspring
        pop[:] = offspring

        # statistics and hall of fame
        record = stats.compile(pop)
        logbook.record(gen=i+1, nevals=len(invalid_ind), imm=len(boring_inds), **record)
        hof.update(pop)
        print(logbook.stream)


        ## save results in checkpoint in case of problems
        cp = dict(population=pop, generation=i, halloffame=hof,
                      logbook=logbook, rndstate=random.getstate())

        with open("checkpoint_%d.pkl" % (i+1), "wb") as cp_file:
            pickle.dump(cp, cp_file)
        



    # finally also save last generation    
    with torch.no_grad():
        new_genes = torch.tensor(pop)
        new_genes = new_genes.view(MU,nz,1,1)
        fake = netG(new_genes.float()).detach().cpu()
        save_image(fake.data, img_save_path + '/evolution_collaborative_%d.png' % (i+1), normalize=True, nrow = 5)

    return pop, logbook, hof

In [ ]:
pop, log, hof = complex_Evolution()

In [ ]:
# Check whether after evolution genes still fit expected distribution
avg = []
std = []
for i,ind in enumerate(pop):
    avg.append(np.mean(ind))
    std.append(np.std(ind))
print(np.mean(avg))
print(np.mean(std))

In [ ]:
# write Logbook as csv
gen = []
nevals = []
imm = []
maxi = []
avg = []
mini = []
std = []

for entry in log:
    gen.append(entry['gen'])
    nevals.append(entry['nevals'])
    imm.append(entry['imm'])
    maxi.append(entry['max'])
    avg.append(entry['avg'])
    mini.append(entry['min'])
    std.append(entry['std'])

import csv
from itertools import zip_longest
d = [gen, nevals, imm, maxi, avg, mini, std]
export_data = zip_longest(*d, fillvalue = '')
with open('evolution_simulation_results.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(("gen", "nevals", "immigrants","max", "avg", "min", "std"))
    wr.writerows(export_data)
myfile.close()

In [ ]:
## plot results

# set upper and lower bound of standard error
se = ([x/np.sqrt(15) for x in std])

lower = [a_i - b_i for a_i, b_i in zip(avg, se)]
upper = [a_i + b_i for a_i, b_i in zip(avg, se)]


fig, ax = plt.subplots()
ax.plot(gen, avg, color = 'blue', label = 'mean')
ax.fill_between(gen, lower, upper, alpha=0.2)
ax.plot(gen, maxi, color = 'darkblue', label = 'max', linestyle='dashed', linewidth = 0.5)
ax.plot(gen, mini, color = 'lightblue', label = 'min', linestyle='dashed', linewidth = 0.5)

ax.set_ylabel('Fitness')
ax.set_xlabel("Generation")
ax.legend(loc = 'upper left')

plt.show()

In [ ]:
## upscale hall of fame
from high_resolution import upscale

for i, img in enumerate(hof):

    # save images
    with torch.no_grad():
        new_genes = torch.tensor(img)
        new_genes = new_genes.view(1,nz,1,1)
        fake = netG(new_genes.float()).detach().cpu()
        save_image(fake.data, img_save_path + '/evolution_collaborative_hall_of_fame_%d.png' % (i), normalize=True)


    upscale(img_path = img_save_path + '/evolution_collaborative_hall_of_fame_%d.png' % (i), scale = 8)

In [ ]:
# Visualize Evolution

# if checkpointing, have to read in manually
from matplotlib import image

img_list = []

for i in range(25):
    img = image.imread(img_save_path + 'evolution_collaborative_%d.png' % (i))
    img_list.append(img)

import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
fig = plt.figure(figsize=(8,6))
plt.axis("off")
ims = [[plt.imshow(i, animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [ ]:
# plot hall of fame

img_list = []

for i in range(20):
    img = image.imread(img_save_path + '/evolution_collaborative_hall_of_fame_%d.png' % (i))
    img_list.append(torch.from_numpy(img))

img_list = torch.stack(img_list, dim=0)

# Plot some training images
plt.figure(figsize=(12,14))
plt.axis("off")
plt.title("Hall of Fame")
plt.imshow(np.transpose(vutils.make_grid(np.transpose(img_list.to(device), (0,3,1,2)), padding=2, normalize=True, nrow = 5).cpu(),(1,2,0)))